In [1]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from deeptime.models.multiclass.linear import LinearClassifier
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset

In [2]:
class LitProgressBar(pl.callbacks.TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(
            disable=True,
        )
        return bar

In [3]:
DATASET_LIST = []
ACC_MEAN_LIST = []
ACC_STD_LIST = []
ACTIVATION_LIST = []

In [5]:
DATASETS = [
    # 'Adiac',
    'ECG5000',
    # 'ChlorineConcentration',
    # 'DistalPhalanxOutlineAgeGroup',
    # 'DistalPhalanxTW',
    # 'ElectricDevices',
]
ACTIVATION = ['relu', 'sinelu', 'leakysinelu']
NUM_EXPERIMENTS = 5

In [10]:
for dataset in DATASETS:
    print()
    print('=' * 50)
    print(f'Executing experiment with dataset {dataset}')

    for activation_fn in ACTIVATION:
        
        accs = []
        for _ in range(NUM_EXPERIMENTS):
            x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
            y_train = np.array(y_train, dtype=np.int32)
            
            NUM_CLASSES = len(np.unique(y_train))
            
            y_train = y_train - 1
            y_train = np.eye(NUM_CLASSES)[y_train]

            sequence_length = x_train.values[0][0].shape[0]

            x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
            y_test = np.array(y_test, dtype='int32') # Fixing the labels type
            y_test = y_test - 1
            y_test = np.eye(NUM_CLASSES)[y_test]

            x_train_transformed = []
            for val in x_train.values:
                x_train_transformed.append(val[0].tolist())
            x_train = np.array(x_train_transformed)

            x_test_transformed = []
            for val in x_test.values:
                x_test_transformed.append(val[0].tolist())
            x_test = np.array(x_test_transformed)

            full_train_dataset = BaseDataset(x=x_train, y=y_train)
            train_loader = DataLoader(full_train_dataset, batch_size=32)

            test_dataset = BaseDataset(x=x_test, y=y_test)
            test_loader = DataLoader(test_dataset, batch_size=32)

            model = LinearClassifier(
                in_features=sequence_length,
                num_classes=NUM_CLASSES,
                activation=ACTIVATION
            )
    
            trainer = pl.Trainer(
                max_epochs=100,
                accelerator='gpu',
                devices=-1,
                callbacks=[LitProgressBar()]
            )

            trainer.fit(model, train_dataloaders=train_loader)
            results = trainer.test(model, test_loader)[0]
            accs.append(results['Accuracy Score'])

        accs = np.array(accs)
        mean, std = accs.mean(), accs.std()
        
        DATASET_LIST.append(dataset)
        ACC_MEAN_LIST.append(mean)
        ACC_STD_LIST.append(std)
        ACTIVATION_LIST.append(activation_fn)



--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 13.5 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
13.5 K    Trainable params
0         Non-trainable params
13.5 K    Total params
0.054     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 47.59it/s, loss=1.05, v_num=67, train_loss=0.975]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 46.49it/s, loss=1.05, v_num=67, train_loss=0.975]

...

Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 50.13it/s, loss=1.04, v_num=69, train_loss=0.967]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████| 141/141 [00:01<00:00, 138.69it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.9224444444444444     │
└───────────────────────────┴───────────────────────────┘

Accuracy: [0.9124444444444444, 0.9177777777777778, 0.9224444444444444]


In [11]:
ACTIVATION = 'lisin'

for dataset in DATASETS:
    print()
    print('-'*50)
    print(f'Executing experiment with dataset {dataset}')
    accs = []

    for _ in range(NUM_EXPERIMENTS):
        x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
        y_train = np.array(y_train, dtype=np.int32)
        y_train = y_train - 1
        y_train = np.eye(NUM_CLASSES)[y_train]

        sequence_length = x_train.values[0][0].shape[0]

        x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
        y_test = np.array(y_test, dtype='int32') # Fixing the labels type
        y_test = y_test - 1
        y_test = np.eye(NUM_CLASSES)[y_test]

        x_train_transformed = []
        for val in x_train.values:
            x_train_transformed.append(val[0].tolist())
        x_train = np.array(x_train_transformed)

        x_test_transformed = []
        for val in x_test.values:
            x_test_transformed.append(val[0].tolist())
        x_test = np.array(x_test_transformed)

        full_train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(full_train_dataset, batch_size=32)

        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        model = LinearClassifier(
            in_features=sequence_length,
            num_classes=NUM_CLASSES,
            activation=ACTIVATION
        )
    
        trainer = pl.Trainer(
            max_epochs=100,
            accelerator='gpu',
            devices=-1,
            callbacks=[LitProgressBar()]
        )

        trainer.fit(model, train_dataloaders=train_loader)
        results = trainer.test(model, test_loader)[0]
        accs.append(results['Accuracy Score'])

    print(f"Accuracy: {accs}")


--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 13.5 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
13.5 K    Trainable params
0         Non-trainable params
13.5 K    Total params
0.054     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 47.67it/s, loss=1.04, v_num=70, train_loss=0.975]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 44.85it/s, loss=1.04, v_num=70, train_loss=0.975]


...

Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 59.61it/s, loss=1.05, v_num=72, train_loss=0.983]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████| 141/141 [00:01<00:00, 125.26it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.9222222222222223     │
└───────────────────────────┴───────────────────────────┘

Accuracy: [0.9148888888888889, 0.9202222222222223, 0.9222222222222223]


In [13]:
ACTIVATION = 'lisilu'

for dataset in DATASETS:
    print()
    print('-'*50)
    print(f'Executing experiment with dataset {dataset}')
    accs = []

    for _ in range(NUM_EXPERIMENTS):
        x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
        y_train = np.array(y_train, dtype=np.int32)
        y_train = y_train - 1
        y_train = np.eye(NUM_CLASSES)[y_train]

        sequence_length = x_train.values[0][0].shape[0]

        x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
        y_test = np.array(y_test, dtype='int32') # Fixing the labels type
        y_test = y_test - 1
        y_test = np.eye(NUM_CLASSES)[y_test]

        x_train_transformed = []
        for val in x_train.values:
            x_train_transformed.append(val[0].tolist())
        x_train = np.array(x_train_transformed)

        x_test_transformed = []
        for val in x_test.values:
            x_test_transformed.append(val[0].tolist())
        x_test = np.array(x_test_transformed)

        full_train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(full_train_dataset, batch_size=32)

        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        model = LinearClassifier(
            in_features=sequence_length,
            num_classes=NUM_CLASSES,
            activation=ACTIVATION
        )
    
        trainer = pl.Trainer(
            max_epochs=100,
            accelerator='gpu',
            devices=-1,
            callbacks=[LitProgressBar()]
        )

        trainer.fit(model, train_dataloaders=train_loader)
        results = trainer.test(model, test_loader)[0]
        accs.append(results['Accuracy Score'])

    print(f"Accuracy: {accs}")


--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 13.5 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
13.5 K    Trainable params
0         Non-trainable params
13.5 K    Total params
0.054     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 54.44it/s, loss=1.45, v_num=73, train_loss=1.320]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 49.76it/s, loss=1.45, v_num=73, train_loss=1.320]

...

Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 47.22it/s, loss=1.45, v_num=75, train_loss=1.320]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████| 141/141 [00:01<00:00, 119.66it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.5837777777777777     │
└───────────────────────────┴───────────────────────────┘

Accuracy: [0.5837777777777777, 0.5837777777777777, 0.5837777777777777]


In [4]:
ACTIVATION = 'leakysinelu'

for dataset in DATASETS:
    print()
    print('-'*50)
    print(f'Executing experiment with dataset {dataset}')
    accs = []

    for _ in range(NUM_EXPERIMENTS):
        x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
        y_train = np.array(y_train, dtype=np.int32)
        y_train = y_train - 1
        y_train = np.eye(NUM_CLASSES)[y_train]

        sequence_length = x_train.values[0][0].shape[0]

        x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
        y_test = np.array(y_test, dtype='int32') # Fixing the labels type
        y_test = y_test - 1
        y_test = np.eye(NUM_CLASSES)[y_test]

        x_train_transformed = []
        for val in x_train.values:
            x_train_transformed.append(val[0].tolist())
        x_train = np.array(x_train_transformed)

        x_test_transformed = []
        for val in x_test.values:
            x_test_transformed.append(val[0].tolist())
        x_test = np.array(x_test_transformed)

        full_train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(full_train_dataset, batch_size=32)

        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        model = LinearClassifier(
            in_features=sequence_length,
            num_classes=NUM_CLASSES,
            activation=ACTIVATION
        )
    
        trainer = pl.Trainer(
            max_epochs=100,
            accelerator='gpu',
            devices=-1,
            callbacks=[LitProgressBar()]
        )

        trainer.fit(model, train_dataloaders=train_loader)
        results = trainer.test(model, test_loader)[0]
        accs.append(results['Accuracy Score'])

    print(f"Accuracy: {accs}")


--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 13.5 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
13.5 K    Trainable params
0         Non-trainable params
13.5 K    Total params
0.054     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 56.02it/s, loss=1.05, v_num=76, train_loss=0.979]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 53.39it/s, loss=1.05, v_num=76, train_loss=0.979]

...

Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 45.33it/s, loss=1.04, v_num=78, train_loss=0.969]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████| 141/141 [00:01<00:00, 112.13it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.9242222222222222     │
└───────────────────────────┴───────────────────────────┘

Accuracy: [0.9088888888888889, 0.9175555555555556, 0.9242222222222222]
